In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit


In [27]:
x_data = np.array(
    [
        0.0,
        0.15789474,
        0.31578947,
        0.47368421,
        0.63157895,
        0.78947368,
        0.94736842,
        1.10526316,
        1.26315789,
        1.42105263,
        1.57894737,
        1.73684211,
        1.89473684,
        2.05263158,
        2.21052632,
        2.36842105,
        2.52631579,
        2.68421053,
        2.84210526,
        3.0,
    ]
)
y_data = np.array(
    [
        2.95258285,
        2.49719803,
        -2.1984975,
        -4.88744346,
        -7.41326345,
        -8.44574157,
        -10.01878504,
        -13.83743553,
        -12.91548145,
        -15.41149046,
        -14.93516299,
        -13.42514157,
        -14.12110495,
        -17.6412464,
        -16.1275509,
        -16.11533771,
        -15.66076021,
        -13.48938865,
        -11.33918701,
        -11.70467566,
    ]
)


In [28]:
plt.figure()
plt.plot(x_data,y_data,'o')
plt.show()

Here is a model where $\beta=(a,b,c)$ and
$$ f(x,\beta)=f(x,a,b,c)=a(x-b)^2+c $$
We fit to find the optimal values of $ a,b,c $ that fit the data above.

In [29]:
# define the model function
def model_f(x, b, a, c):
    return a * (x - b) ** 2 + c


# use scipy curve_fit function
initial_guess = [3, 2, -16]  # a,b,c
popt, pcov = curve_fit(model_f, x_data, y_data, p0=initial_guess)


In [30]:
popt # contain optimum a,b,c values

array([  2.00492121,   4.92207992, -16.04690266])

pcov: the covariance matrix, which gives an estimate of the "error" of the parameters (based on how sensitive the "goodness-of-fit" is to changing them) and the relation the parameters have to eachother (are two seperate parameters really needed?)

In [31]:
pcov

array([[ 0.00222114, -0.01315915,  0.00322289],
       [-0.01315915,  0.12827822, -0.07363194],
       [ 0.00322289, -0.07363194,  0.13377697]])

In [32]:
a_opt, b_opt, c_opt = popt
x_model = np.linspace(np.min(x_data), np.max(x_data), 100)
y_model = model_f(x_model, a_opt, b_opt, c_opt)

plt.figure()
plt.plot(x_data,y_data,'o')
plt.plot(x_model, y_model, 'r.')
plt.show()


pcov is difficult to interpret, but we can do so as follows.

the "error" of a parameter in the model is related to how sensitive the model is to that parameter. Does changing the parameter slightly give a better fit? (i.e. minimize $\sum{(f(x_i,\beta)-y_i)^2}$ )

In [33]:
def fit_f(x, a, b, c, d):
    return a * (x - b) ** 2 + c + d * 0.0001 * np.cos(x)


initial_guess = [1, 2, -16, 1]  # a,b,c,d
popt, pcov = curve_fit(fit_f, x_data, y_data, p0=initial_guess)


In [34]:
pcov

array([[ 1.45024174e-01, -4.03732594e-02,  4.08922441e-02,
         3.05314996e+03],
       [-4.03732594e-02,  7.20537355e-02, -3.07488421e-01,
        -8.46751267e+03],
       [ 4.08922441e-02, -3.07488421e-01,  1.52776232e+00,
         3.78842426e+04],
       [ 3.05314996e+03, -8.46751267e+03,  3.78842426e+04,
         1.03326934e+09]])

In [35]:
plt.figure()
plt.imshow(np.log(np.abs(pcov)))
plt.colorbar()
plt.show()

# see, the error on d is high and covarance is also high
# so, changing it does not have a big effect on the model
# get rid of it!


# Examples

In [36]:
# fitting a gausian curve

df = pd.read_csv("data/curvefite1.csv")
x_stuff = df.Channel
y_stuff = df.Cs137


In [37]:
plt.figure()
plt.plot(x_stuff, y_stuff, ".")
plt.show()


This curve represents the photon spectrum of Cesium 137 (a common radioactive substance). The main curve represents the primary photon emission peak.

- The mean value of the peak tells us about the energy of photons emitted.
- The Width tells us about the resolution of the detector.

Typically we fit peaks like this to Gaussian curves and use $\mu$ and $\sigma$ to extract information about the two points above.

$$ f(x,A,\mu,\sigma)=Ae^{-(x-\mu^2)/\sigma^2} $$


In [38]:
def gauss_f(x, A, mu, sig):
    return A * np.exp(-((x - mu) ** 2) / sig**2)


x_data = x_stuff[(x_stuff > 725) & (x_stuff < 850)]
y_data = y_stuff[(x_stuff > 725) & (x_stuff < 850)]

plt.figure()
plt.plot(x_data, y_data, ".g")
plt.show()


In [39]:
p_opt, p_cov = curve_fit(gauss_f, x_data, y_data, p0=[2000, 790, 60])

A_opt, mu_opt, sig_opt = p_opt
x_model = np.linspace(min(x_data), max(x_data), 1000)
y_model = gauss_f(x_model, A_opt, mu_opt, sig_opt)

plt.figure()
plt.plot(x_model, y_model, ".r")
plt.plot(x_data, y_data, ".g")
plt.show()


In [40]:
p_opt

array([1939.55510385,  786.33851861,   57.78717218])

In [41]:
plt.figure()
plt.imshow(np.log(np.abs(p_cov)))
plt.colorbar()
plt.show()


In [42]:
np.sqrt(np.diag(p_cov))

# small errors on the parameters - that is great!


array([5.70257763, 0.14401935, 0.26427165])

Now suppose the data above has errors given by 
$δ_{y_i}=\sqrt{y_i}$ for each data point $y_i$. (These are the true errors for a counting experiment).

In [59]:
yerr_data = np.sqrt(y_data)

plt.figure()
plt.plot(x_data, y_data, "-k")
# plt.errorbar(x_data,y_data,yerr_data, ls='', color='k')
plt.fill_between(x_data, y_data - yerr_data, y_data + yerr_data, color="m", alpha=0.3)
plt.show()


In [63]:
# use sigma to get the maximum likelihood estimator
popt, pcov = curve_fit(gauss_f, x_data, y_data, sigma=yerr_data, p0=[2000, 790, 60])
popt


array([1936.83040984,  786.29515455,   57.88161227])

In [65]:
# Leonard Weibeck potential

r_data, V_data = np.loadtxt("data/curvefite2.txt")


$$V(r)=4 \epsilon \left[(\frac{\sigma}{r})^12-(\frac{\sigma}{r})6\right]$$

In [73]:
plt.figure()
plt.plot(r_data, V_data, "ok")
plt.xlabel("$r$ [m]", fontsize=20)
plt.ylabel("$V(r)$ [J]", fontsize=20)
plt.tight_layout()
plt.show()


First its probably a good idea to standardize our units. We can convert back after:

- Multiply x-axis by 1e9. This converts all quantities with dimenions of meters in this problem to nanometers

- Divide y-axis by Boltzmann constant $kB = 1.3806452 × 10−23 J/K$ This converts all quantites with dimensions of joules to Kelvin.

In [74]:
r_data *= 1e9
V_data /= 1.38e-23


In [76]:
plt.figure()
plt.plot(r_data, V_data, "ok")
plt.xlabel("$r$ [nm]", fontsize=20)
plt.ylabel("$V(r)$ [K]", fontsize=20)
plt.tight_layout()
plt.show()


In [77]:
def model_f(r, sigma, epsilon):
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)


eps_init_guess = 120
sig_init_guess = 2 ** (-1 / 6) * 0.4  # from wikipedia

popt, pcov = curve_fit(model_f, r_data, V_data, p0=[sig_init_guess, eps_init_guess])

sig_opt, eps_opt = popt

r_model = np.linspace(min(r_data), max(r_data), 1000)
V_model = model_f(r_model, sig_opt, eps_opt)

plt.figure()
plt.plot(r_data, V_data, "ok")
plt.plot(r_model, V_model, "-r")
plt.xlabel("$r$ [nm]", fontsize=20)
plt.ylabel("$V(r)$ [K]", fontsize=20)
plt.tight_layout()
plt.show()



In [78]:
sig_opt

0.34143879559186735

In [81]:
sig_init_guess

0.35635948725613575

In [79]:
eps_opt

124.98902618753002

In [80]:
eps_init_guess

120